# Mentoría
## Análisis y predicción de distribución troncal de energía

### Introducción

##### Análisis y Visualización
En la siguiente notebook, realizaremos la carga del dataset que utilizarán para el práctico de la materia. Para ello, comenzaremos con las importaciones pertinentes.
Posteriormente, se detallarán las consignas a seguir para la elaboración del informe.


In [4]:
# Importación de las librerías necesarias
import numpy as np
import pandas as pd
import matplotlib as mpl
mpl.get_cachedir()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import datetime as dt
import warnings

In [5]:
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', 150)
pd.set_option('max_colwidth', 151)
warnings.filterwarnings('ignore')

## Revisión general de los datos

In [10]:
#Parsing auxiliar
dateparse = lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')

In [38]:
# _ds_energía
_ds_energia = pd.read_csv('https://raw.githubusercontent.com/alaain04/diplodatos/master/data/distribucion.csv',
                          dtype={'Amper fase T-A': float},
                          parse_dates=['Fecha'],
                          date_parser=dateparse,
                          float_precision='round_trip')

### DS Energía

In [40]:
_ds_energia.head(5)

,Fecha,Amper fase T-A,Amper fase S-A,Amper fase R-A,Vab,Vca,Vbc,Kwatts 3 fases,Factor de Poten-A
0,2019-12-03 20:45:00,11.0,11.0,12.0,2147483.65,42.16,40.94,-2.147484e+09,0.0
1,2019-12-03 20:45:00,11.0,11.0,12.0,2147483.65,42.16,40.94,-2.147484e+09,0.0
2,2019-12-03 20:45:00,11.0,11.0,12.0,2147483.65,42.16,40.94,-2.147484e+09,0.0
3,2019-12-03 20:45:00,11.0,11.0,12.0,2147483.65,42.16,40.94,-2.147484e+09,0.0
4,2019-12-03 20:45:00,11.0,11.0,12.0,2147483.65,42.16,40.94,-2.147484e+09,0.0


In [28]:
data_types = _ds_energia.dtypes
data_types = pd.DataFrame(data_types)
data_types = data_types.rename(columns = {0:'ReadingDataType'})
data_types.index.names = ['FieldName']
data_types

,ReadingDataType
FieldName,
Fecha,datetime64[ns]
Amper fase T-A,float64
Amper fase S-A,float64
Amper fase R-A,float64
Vab,float64
Vca,float64
Vbc,float64
Kwatts 3 fases,float64
Factor de Poten-A,float64


In [43]:
# Descripción
_ds_energia.describe(include = 'all')

,Fecha,Amper fase T-A,Amper fase S-A,Amper fase R-A,Vab,Vca,Vbc,Kwatts 3 fases,Factor de Poten-A
count,83518,83518.000000,83517.000000,83500.000000,7.452300e+04,7.452300e+04,7.452300e+04,8.265400e+04,83408.000000
unique,27434,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,2019-12-26 10:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
first,2019-12-03 20:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,2020-05-07 11:10:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,6.549163,6.595879,6.823796,3.513603e+05,3.529752e+05,3.534354e+05,-3.171852e+08,-0.339098
std,NaN,7.545152,7.569962,7.749175,7.943730e+05,7.958370e+05,7.962548e+05,7.619362e+08,0.779093
min,NaN,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,-2.147484e+09,-0.980000
25%,NaN,0.000000,0.000000,0.000000,3.527000e+01,3.635000e+01,3.557000e+01,-5.272000e+03,-0.930000


#### Aclaraciones
* Features
    * **Fecha**: Dia/hora de la medición.
    * **Amper fase T-A, Amper fase S-A, Amper fase R-A**: Corrientes en cada una de las fases.
    * **Vab, Vca, Vbc**: Tensión relativa entre las fases.
    * **Kwatts 3 fases**: Potencia total.
    * **Factor de Potencia**: Constante.
    
* Términos
    * Corriente = intensidad [Amperes] 
    * Voltaje = tensión [Volts]
    * Potencia total [Kw]
    * Factor de potencia = Φ
    * Corriente total = Promedio de corrientes entre fases.
    * Tensión total = Promedio de tensiones entre fases.
    

### DS Energía

In [42]:
# _ds_clima
_ds_clima = pd.read_csv('https://github.com/alaain04/diplodatos/raw/master/data/clima.csv',
                        parse_dates=['time'],
                        date_parser=dateparse)

In [34]:
_ds_clima.head(5)

,time,temperature,dewpoint,humidity,precipitation,precipitation_3,precipitation_6,snowdepth,windspeed,peakgust,winddirection,pressure,condition
0,2019-11-01 00:00:00,20.1,18.0,88,NaN,NaN,NaN,NaN,6.1,NaN,125.0,1014.3,NaN
1,2019-11-01 01:00:00,19.5,17.8,90,NaN,NaN,NaN,NaN,5.4,NaN,106.0,1014.9,NaN
2,2019-11-01 02:00:00,19.2,17.7,91,NaN,NaN,NaN,NaN,4.3,NaN,83.0,1015.6,NaN
3,2019-11-01 03:00:00,18.7,17.7,94,NaN,NaN,NaN,NaN,4.3,NaN,87.0,1015.7,NaN
4,2019-11-01 04:00:00,18.0,17.0,94,NaN,NaN,NaN,NaN,6.1,NaN,91.0,1016.2,NaN


In [35]:
data_types = _ds_clima.dtypes
data_types = pd.DataFrame(data_types)
data_types = pd.DataFrame(data_types)
data_types = data_types.rename(columns = {0:'ReadingDataType'})
data_types.index.names = ['FieldName']
data_types

,ReadingDataType
FieldName,
time,datetime64[ns]
temperature,float64
dewpoint,float64
humidity,int64
precipitation,float64
precipitation_3,float64
precipitation_6,float64
snowdepth,float64
windspeed,float64


In [44]:
_ds_clima.describe(include = 'all')

,time,temperature,dewpoint,humidity,precipitation,precipitation_3,precipitation_6,snowdepth,windspeed,peakgust,winddirection,pressure,condition
count,3001,3001.000000,3001.000000,3001.000000,0.0,0.0,0.0,0.0,2999.000000,0.0,2999.000000,3001.000000,0.0
unique,3001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,2020-01-30 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
first,2019-11-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,2020-05-13 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,23.018294,15.841353,67.565811,NaN,NaN,NaN,NaN,6.595599,NaN,144.830944,1012.926891,NaN
std,NaN,5.485819,4.508826,19.787111,NaN,NaN,NaN,NaN,3.795323,NaN,88.453037,4.166595,NaN
min,NaN,4.700000,1.100000,18.000000,NaN,NaN,NaN,NaN,0.400000,NaN,1.000000,1001.100000,NaN
25%,NaN,19.700000,12.600000,53.000000,NaN,NaN,NaN,NaN,3.600000,NaN,87.000000,1010.200000,NaN


#### Aclaraciones
* API: https://api.meteostat.net
* Features
    * **time**: Día/hora de la medición. 
    * **temperature**: Temperatura [Celcius]
    * **dewpoint**: Punto de rocío [mm]
    * **humedity**: Humedad.
    * **precipitation, precipitation_3, precipitation_6**: Precipitación [mm]
    * **snowdepth**: Nieve caida [mm]
    * **winddirection**: Dirección del viento [Grados]
    * **pressure**: Presión atmosférica [mmHg]
    * **condition**: Condición [string]


###### Extra Tip
Pandas permite indexar el dataframe con el tipo PeriodIndex, el cual podemos utilizarlos para agrupar registros según la frecuancia definida (en este ejemplo 30 minutos).

```_data.index = pd.PeriodIndex(list(_data['Fecha']), freq='T')
_pt_period_mean = _data['Kwatts 3 fases'].resample('30T').mean()```

## Análisis y Visualización - Consigna

### General
Se propone la elaboración de un informe que deberá entregarse en formato html, pdf o markdown y deberá apuntar a un público técnico, pero sin conocimientos específicos en el tema. Añadir gráficos y conclusiones obtenidas durante el desarrollo del práctico.

### Análisis descriptivo
- Entender qué tipos de features poseen los dataset, cómo se relacionan entre ellos.
- Identificar features objetivo. 

### Análisis estadístico
- Ordenar temporalmente los dataset.
- Seleccionar al menos 3 features relevantes para el problema. Utilizar los gráficos y cálculos que le parezcan más apropiados para llegar a las conclusiones.

    - ¿Existen outliers para estos features?
    - ¿Hay valores faltantes?
    - ¿Hay valores negativos?
    - ¿Que decisión tomarían para completar o eliminar estos outliers y faltantes?
    - ¿Cómo es la distribución de las variables en el dataset?
    - ¿Se observa alguna estacionalidad?

- ¿Existe correlación entre las variables?
- ¿Que variables se correlacionan más con la potencia distribuida?

- Calidad del servicio: 
    - Frecuencia de corte del suministro. 
    - Duración de los cortes.
    - Franjas temporales con mayores cortes.
- Causas de corte de servicio:
    - Analizar que sucede con los demás features al momento de una corte.
    - Se observa alguna relación con la temperatura y velocidad/dirección del viento en estos períodos?
     
- Analizar el comportamiento de la distribución semalmente, es comparable mes a mes? La distribución de potencia, ¿se asemeja a alguna conocida?


### Preguntas adicionales
- ¿Cómo se comporta la distribución de energía ? ¿Qué relaciones observamos con los usos y costumbres del lugar? ¿Y con las condiciones atmosféricas?
- ¿Por qué se generan cortes en la distribución? ¿Se observa algún comportamiento anterior que nos anticipen estas caídas? 
- ¿Existen factores estacionales que influyen en las cargas de distribución? ¿Hubo algún cambio del comportamiento de consumo debido a la pandemia o en fechas festivas(navidad, año nuevo, etc)?
- ¿Qué información podemos incorporar a la planificación de mantenimientos de la línea?
